### 목록
* 준비
    * [사용할 라이브러리 불러오기](#lib)
    * [엘라스틱서치 환경 설정하기](#es_setting)
    * [엘라스틱서치에서 데이터 조회하기](#es_load)
    * [조회한 데이터를 테이블 형태로 만들기](#df)
* Bucket Aggregation
    * [Date Histogram Aggregation](#dh)
    * [Date Range Aggregation](#dr)
    * [Histogram Aggregation](#ha)
    * [Range Aggregation](#ra)
    * [Terms Aggregation](#ta)
    * [Filters Aggregation](#fa)
---

### 준비

<a name='lib'></a>
#### 사용할 라이브러리 불러오기

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import datetime

<a name='es_setting'></a>
#### 엘라스틱서치 환경 설정하기

In [2]:
# 적당한 값을 넣지 않으면 오류가 납니다
es = Elasticsearch()
es_index = 'test_index'
es_type = 'test_type'

<a name='es_load'></a>
#### 엘라스틱서치에서 데이터 조회하기

In [3]:
x = es.search(
      index='test_index', 
      doc_type='test_type', 
      body={
        '_source' : ['결제카드','고객나이', '구매사이트', '상품가격', '주문시간'],
        'query' : {
            'match_all' : {}
        },
        'size' : 50
      }
)

<a name='df'></a>
#### 조회한 데이터를 테이블 형태로 만들기

In [4]:
df = pd.DataFrame([x['hits']['hits'][idx]['_source'] for idx, _ in enumerate(x['hits']['hits'])])
df

,결제카드,고객나이,구매사이트,상품가격,주문시간
0,국민,28,옥션,8000,2017-11-01T05:45:01
1,하나,32,티몬,13000,2017-11-09T16:58:07
2,국민,35,위메프,22000,2017-11-06T20:03:26
3,국민,38,GS샵,21000,2017-11-03T18:29:35
4,롯데,28,쿠팡,21000,2017-11-09T22:37:36
5,신한,30,티몬,13000,2017-11-05T00:41:55
6,신한,41,옥션,15000,2017-11-06T00:27:44
7,국민,38,쿠팡,12000,2017-11-09T00:40:16
8,우리,39,11번가,10000,2017-11-06T10:28:26
9,국민,39,GS샵,17000,2017-11-08T02:00:16


### Bucket Aggregation

<a name='dh'></a>
#### Date Histogram Aggregation

: Date Field를 일정 간격으로 나눈 구간으로 Bucket(혹은 Group) 생성

In [5]:
new_df = df.copy()
new_df.index = pd.to_datetime(new_df['주문시간'])
new_df_rs = new_df.resample('D').count().iloc[:, 1:2]
new_df_rs = pd.DataFrame({'주문시간' : new_df_rs.index, '개수' : new_df_rs.values.ravel()})
new_df_rs[['주문시간', '개수']]

,주문시간,개수
0,2017-11-01,2
1,2017-11-02,4
2,2017-11-03,5
3,2017-11-04,7
4,2017-11-05,4
5,2017-11-06,6
6,2017-11-07,5
7,2017-11-08,4
8,2017-11-09,8
9,2017-11-10,5


<a name="dr"></a>
#### Date Range Aggregation
:  각 Date 구간의 시작과 끝점을 설정하여 Bucket 생성한다

##### 11월1일 ~ 11월3일 구간

In [6]:
new_df_rs = new_df['2017-11-01' : '2017-11-03'].resample('D').count().iloc[:, 1:2]
new_df_rs = pd.DataFrame({'주문시간' : new_df_rs.index, '개수' : new_df_rs.values.ravel()})
new_df_rs[['주문시간', '개수']]

,주문시간,개수
0,2017-11-01,2
1,2017-11-02,4
2,2017-11-03,5


In [7]:
new_df['2017-11-01' : '2017-11-03'].count()[0]

11

##### 11월4일 ~ 11월10일 구간

In [8]:
new_df_rs = new_df['2017-11-04' : '2017-11-10'].resample('D').count().iloc[:, 1:2]
new_df_rs = pd.DataFrame({'주문시간' : new_df_rs.index, '개수' : new_df_rs.values.ravel()})
new_df_rs[['주문시간', '개수']]

,주문시간,개수
0,2017-11-04,7
1,2017-11-05,4
2,2017-11-06,6
3,2017-11-07,5
4,2017-11-08,4
5,2017-11-09,8
6,2017-11-10,5


In [9]:
new_df['2017-11-04' : '2017-11-10'].count()[0]

39

<a name="ha"></a>
#### Histogram Aggregation
: Number Field의 각 구간으로 Bucket 생성한다

In [10]:
bins = range(10, 51, 10)
group_names = ['10대', '20대', '30대', '40대']
df['연령대'] = pd.cut(df['고객나이'], bins, labels=group_names)
df_gb = df.groupby(by='연령대').count().iloc[:, 1:2]
df_gb = pd.DataFrame({'연령대' : df_gb.index, '개수' : df_gb.values.ravel()})
df_gb[['연령대', '개수']]

,연령대,개수
0,10대,6
1,20대,17
2,30대,18
3,40대,9


<a name="ra"></a>
#### Range Aggregation
: 옵션으로 설정한 Number Field의 각 구간으로 Bucket 생성한다

* 상품가격이 만원 이하 : [0, 10,000)
* 상품가격이 이만원 이하 : [10,000 , 20000)
* 상품가격이 삼만원 이하 : [20,000 , 30000)

In [11]:
bins = range(0, 30001, 10000)
group_names = ['만원 이하', '이만원 이하', '삼만원 이하']
df['가격대'] = pd.cut(df['상품가격'], bins, labels=group_names)
df_gb = df.groupby(by='가격대').count().iloc[:, 1:2]
df_gb = pd.DataFrame({'가격대' : df_gb.index, '개수' : df_gb.values.ravel()})
df_gb[['가격대', '개수']]

,가격대,개수
0,만원 이하,13
1,이만원 이하,20
2,삼만원 이하,17


<a name="ta"></a>
#### Terms Aggregation
: 선택한 Term를 기준으로 Bucket 생성한다

* 카드별 결제 건수가 
* 가장 많은 2개의 카드 별
* 최대 결제 금액을 구하시오

In [12]:
# 카드별 결제 건수가...
card = df.groupby(by='결제카드').count().iloc[:, 1:2]
card_df = pd.DataFrame({'결제카드' : card.index, '개수' : card.values.ravel()})
card_df[['결제카드', '개수']]

,결제카드,개수
0,국민,13
1,롯데,6
2,삼성,1
3,시티,1
4,신한,8
5,우리,14
6,하나,7


In [13]:
# ,,,가장 많은 2개의 카드...
card_sort = card_df.sort_values(by='개수', ascending=False)[:2]
card_sort[['결제카드', '개수']]

,결제카드,개수
5,우리,14
0,국민,13


In [14]:
# ...최대 결제 금액을 구하시오
max(df[df['결제카드'] == '우리']['상품가격'])

27000

In [15]:
# ...최대 결제 금액을 구하시오
max(df[df['결제카드'] == '국민']['상품가격'])

26000

<a name="fa"></a>
#### Filters Aggregation
: Query String으로 작성한 조건 만족하는 Bucket 생성한다

* 구매 사이트가 옥션, 티몬인 사람들의
* 구매 사이트별 최소 구매가격을 구하시오

In [16]:
# 구매 사이트가 옥션, 티몬인 사람들의...
df_filter = df[df['구매사이트'].str.contains('옥션|티몬')]
df_filter

,결제카드,고객나이,구매사이트,상품가격,주문시간,연령대,가격대
0,국민,28,옥션,8000,2017-11-01T05:45:01,20대,만원 이하
1,하나,32,티몬,13000,2017-11-09T16:58:07,30대,이만원 이하
5,신한,30,티몬,13000,2017-11-05T00:41:55,20대,이만원 이하
6,신한,41,옥션,15000,2017-11-06T00:27:44,40대,이만원 이하
14,국민,44,옥션,21000,2017-11-04T08:22:51,40대,삼만원 이하
15,신한,20,티몬,28000,2017-11-03T20:15:22,10대,삼만원 이하
18,우리,35,옥션,23000,2017-11-05T04:57:07,30대,삼만원 이하
23,하나,33,옥션,19000,2017-11-06T04:46:55,30대,이만원 이하
32,롯데,41,옥션,27000,2017-11-10T23:15:29,40대,삼만원 이하
33,신한,19,티몬,23000,2017-11-04T20:38:58,10대,삼만원 이하


In [17]:
# ...구매사이트별 최소 구매고객을 구하시오
min(df_filter[df_filter['구매사이트']=='티몬']['상품가격'])

5000

In [18]:
# ...구매사이트별 최소 구매고객을 구하시오
min(df_filter[df_filter['구매사이트']=='옥션']['상품가격'])

8000